In [1]:
import torch
import numpy as np
import copy
import matplotlib.pyplot as plt

from torch.utils.data import Dataset

from utilities import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


/home/qguo0701/.conda/envs/qingguo/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
# batch_size = 20
batch_size = 128

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [24]:
# feature_file = 'features/cca_d_5.npy'
# feature_file = 'features/flo_d_5.npy'
# feature_file = 'features/infonce_d_5.npy'
feature_file = 'feature/fdv_5.npy'

In [22]:
i = 5
feature_file = 'fdv_%d.npy'%i
feature_file

'fdv_5.npy'

In [25]:
data_dict = np.load(feature_file, allow_pickle=True).item()

In [8]:
Z_train = data_dict['Z_train']
Y_train = data_dict['Y_train']
Z_test = data_dict['Z_test']
Y_test = data_dict['Y_test']
method_name = data_dict['method']

dim_z = Z_train.shape[1]
print(dim_z)

14


In [9]:
train_data = list(zip(torch.Tensor(Z_train).to(device), Y_train))
test_data = list(zip(torch.Tensor(Z_test).to(device), Y_test))

In [10]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True, shuffle=False)

In [11]:
import torch.nn as nn
import torch.nn.functional as F
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(512, 512)
        # linear layer (n_hidden -> 10)
#         self.fc3 = nn.Linear(512, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        #x = self.fc3(x)
        return x
    
class SplitEncoder(nn.Module):
    
    '''
    Encoding half of the image
    '''
    def __init__(self):
        super(SplitEncoder, self).__init__()
        self.fc1 = nn.Linear(28 * 14, 512)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(512, 512)
        # linear layer (n_hidden -> 10)
#         self.fc3 = nn.Linear(512, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        # self.dropout = nn.Dropout(0.2)
        self.dropout = nn.Dropout(0.)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        #x = self.fc3(x)
        return x
    
## Define the NN architecture
class Net(nn.Module):
    def __init__(self, encoder):
        super(Net, self).__init__()
        self.encoder = encoder
#         self.fc1 = nn.Linear(28 * 28, 512)
#         # linear layer (n_hidden -> hidden_2)
#         self.fc2 = nn.Linear(512, 512)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(512, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
#         self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
#         x = x.view(-1, 28 * 28)
#         # add hidden layer, with relu activation function
#         x = self.dropout(F.relu(self.fc1(x)))
#         x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(self.encoder(x))
        return x

# # initialize the NN
# encoder = Encoder()
# model = Net(encoder)
# print(model)

model = MLP(dim_z, 10)
model = model.to(device)

In [12]:
## Specify loss and optimization functions

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [13]:
def test_eval(model, test_loader): 
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    model.eval() # prep model for *evaluation*

    for data, target in test_loader:
        
        data = data.to(device)
        target = target.to(device)
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # calculate and print avg test loss
    test_loss = test_loss/len(test_loader.dataset)
#     print('Test Loss: {:.6f}\n'.format(test_loss))

#     for i in range(10):
#         if class_total[i] > 0:
#             print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
#                 str(i), 100 * class_correct[i] / class_total[i],
#                 np.sum(class_correct[i]), np.sum(class_total[i])))
#         else:
#             print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

#     print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
#         100. * np.sum(class_correct) / np.sum(class_total),
#         np.sum(class_correct), np.sum(class_total)))
    return 100. * np.sum(class_correct) / np.sum(class_total), test_loss

In [14]:
# number of epochs to train the model
# n_epochs = 20  # suggest training between 20-50 epochs
# maximal_iterations = int(n_train*50)
# n_epochs = int(maximal_iterations/(n_labeled))
n_epochs = 50
model.train() # prep model for training

counter_list = [0]
test_acc_list = []
train_loss_list = []
test_loss_list = []
counter = 0

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for i, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        counter += batch_size
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    test_acc, test_loss = test_eval(model, test_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}, test loss: {:.6f}, test acc: {:.6f}'.format(
        epoch+1, 
        train_loss, test_loss, test_acc
        ))
    counter_list.append(counter)
    test_acc_list.append(test_acc)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)

Epoch: 1 	Training Loss: 1.403347, test loss: 0.591550, test acc: 81.209936
Epoch: 2 	Training Loss: 0.546752, test loss: 0.514730, test acc: 82.491987
Epoch: 3 	Training Loss: 0.495745, test loss: 0.511756, test acc: 82.431891
Epoch: 4 	Training Loss: 0.473210, test loss: 0.496572, test acc: 83.092949
Epoch: 5 	Training Loss: 0.463110, test loss: 0.492971, test acc: 82.972756
Epoch: 6 	Training Loss: 0.450274, test loss: 0.472504, test acc: 83.964343
Epoch: 7 	Training Loss: 0.441923, test loss: 0.483214, test acc: 83.523638
Epoch: 8 	Training Loss: 0.440295, test loss: 0.490863, test acc: 83.183093
Epoch: 9 	Training Loss: 0.431888, test loss: 0.476032, test acc: 83.683894
Epoch: 10 	Training Loss: 0.429694, test loss: 0.471737, test acc: 84.034455
Epoch: 11 	Training Loss: 0.422749, test loss: 0.475095, test acc: 83.573718
Epoch: 12 	Training Loss: 0.421519, test loss: 0.478717, test acc: 83.633814
Epoch: 13 	Training Loss: 0.411043, test loss: 0.471442, test acc: 83.984375
Epoch: 1

In [15]:
np.amax(np.array(test_acc_list))

84.2948717948718

CCA      67.78
FLO      77.52
InfoNCE  79.27
FDV      79.93
